## Create a Notebook File and perform the following tasks


* Try improving the model and re-code the program from scratch without looking too much at the source code.
* Write-up a summary explaining how your program worksin the markdown cell

In [ ]:
!nvidia-smi

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import time

from IPython import display

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, Conv2D, Flatten, Dropout, MaxPooling2D, Activation,BatchNormalization,LeakyReLU,Dense,Reshape
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import numpy as np

## Dataset Exploration

In [ ]:
(x_train,y_train),(_,_)=mnist.load_data()
x_train.shape

## Preprocessing the dataset

In [ ]:
x_train = (x_train.astype(np.float32) - 127.5)/127.5
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],x_train.shape[2],1)

In [ ]:
x_train.shape

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 300

In [ ]:
x_train = tf.data.Dataset.from_tensor_slices(x_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## Building Generator model

In [ ]:
def generator_model():
    model=Sequential()
    model.add(Dense(7*7*256,input_shape=(100,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(Reshape((7, 7, 256)))

    model.add(Conv2DTranspose(128, (3, 3), strides=(1, 1), padding='same'))

    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(1, (3, 3), strides=(2, 2), padding='same', activation='tanh'))
    return model


In [ ]:
generator = generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')

In [ ]:
generator.summary()

## Building discriminator

In [ ]:
def discriminator_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), strides=(2, 2), padding='same',
                                    input_shape=[28, 28, 1]))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(1))

    return model

In [ ]:
discriminator = discriminator_model()
decision = discriminator(generated_image)
print (decision)

In [ ]:
discriminator.summary()

## Importing loss from tensorflow

In [ ]:
from tensorflow.keras.losses import BinaryCrossentropy
cross_entropy = BinaryCrossentropy(from_logits=True)

## Discriminator loss

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

## Generator Loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

## Defining the optimizer

In [ ]:
from tensorflow.keras.optimizers import Adam
generator_optimizer = Adam(1e-4)
discriminator_optimizer = Adam(1e-4)

In [ ]:
EPOCHS = 400
noise_dim = 100
num_examples_to_generate = 16
BATCH_SIZE = 300
# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

## Train Function

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:

def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    # Produce images for the GIF as we go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    #if (epoch + 1) % 15 == 0:
      #checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [ ]:
train(x_train, EPOCHS)

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('/kaggle/working/image_at_epoch_0100.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('/kaggle/working/image_at_epoch_0200.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('/kaggle/working/image_at_epoch_0250.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('/kaggle/working/image_at_epoch_0400.png')
imgplot = plt.imshow(img)
plt.show()

## Summary

* Initially all the libraries that are required for the given task should be imported.
* Download the MNIST dataset and preprocess it.
* Now create the generator and discriminator model.
* Generator model is the one which produces the fake images and tries to fool the discriminator by the fake images it produced. The discriminator on the other end tries to identify the fake images which are produced by the generator.
* First we need to create the generator model that produces the fake images. We give a random generated input to the generator where it uses the Conv2D transpose layers to produce the images. The upsampling of the image continues until we reach the desired shape.
* Discriminator model is usually trained to classify the images as real or fake i.e., for binary classification. The images from the dataset are marked as real and the images from the generator are marked as fake.
* Discriminator loss depends on how far it is able to distinguish the real images from the fake images.
* The generator loss depends on how far it is able to trick the discriminator.
* When the training starts we give the image predicted from the the generator to the discriminator along with the real images and the discriminator should predict whether it is real or fake. The weights are then updated in the generator and discriminator based on the loss.
* This process continue still the generator is able to fool the discriminator.
* From the plots we can see that our generator is gradually improving.
* We can stop this process when the images are similar to our train images.

